In [1]:
import pandas as pd

df = pd.read_csv('data/others/학생수.csv')
df.head(), df.shape, df['지역별'].unique()


(  지역별    시점    유치원   초등학교    중학교   고등학교
 0  서울  2011  21.81  26.81  32.84  33.57
 1  서울  2012  24.18  25.55  32.01  32.77
 2  서울  2013  24.13  24.51  31.39  31.87
 3  서울  2014  21.87  24.27  30.20  30.94
 4  서울  2015  21.38  24.00  28.49  30.13,
 (236, 6),
 array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object))

In [2]:
df['학생수'] = df['유치원'] + df['초등학교'] + df['중학교'] + df['고등학교']
df = df.drop(columns=['유치원', '초등학교', '중학교', '고등학교'])
df.head(), df.shape, df.columns.unique()

(  지역별    시점     학생수
 0  서울  2011  115.03
 1  서울  2012  114.51
 2  서울  2013  111.90
 3  서울  2014  107.28
 4  서울  2015  104.00,
 (236, 3),
 Index(['지역별', '시점', '학생수'], dtype='object'))

In [3]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'학생수': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['학생수'] = final_monthly_df['학생수'].map(lambda x: f"{x:.3f}")

# 결과 확인
print(final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/education/학생수(월별).csv', index=False)


         시점     학생수 지역별
0   2011.01  99.250  강원
1   2011.02  99.153  강원
2   2011.03  99.057  강원
3   2011.04  98.960  강원
4   2011.05  98.863  강원
5   2011.06  98.767  강원
6   2011.07  98.670  강원
7   2011.08  98.573  강원
8   2011.09  98.477  강원
9   2011.10  98.380  강원
10  2011.11  98.283  강원
11  2011.12  98.187  강원 (2645, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
